## GCP

# Módulo práctico: IA Generativa en Google Cloud Platform

En este módulo práctico vas a trabajar con las capacidades de **Inteligencia Artificial Generativa** de **Google Cloud Platform (GCP)**, utilizando modelos para procesamiento de texto.

## Tendrás acceso a:

- Modelos LLM de la familia **Gemini**, capaces de comprender y generar texto, analizar imágenes y mantener contexto en conversaciones.  
- La **API oficial de Google Generative AI (Gemini API)**.

## Objetivos del módulo:

- Dominar el uso de los principales modelos LLM de **Gemini** y entender sus diferencias.  
- Construir y analizar consultas.  
- Evaluar y comparar respuestas de distintos modelos y configuraciones.  
- Reflexionar sobre la calidad, ética y limitaciones actuales de la IA en producción.


In [1]:
!pip freeze

annotated-types==0.7.0
anyio==4.11.0
asttokens==3.0.1
beautifulsoup4==4.14.2
cachetools==6.2.2
certifi==2025.11.12
charset-normalizer==3.4.4
colorama==0.4.6
comm==0.2.3
debugpy==1.8.17
decorator==5.2.1
executing==2.2.1
google==3.0.0
google-ai-generativelanguage==0.6.15
google-api-core==2.28.1
google-api-python-client==2.187.0
google-auth==2.43.0
google-auth-httplib2==0.2.1
google-cloud==0.34.0
google-genai==1.50.1
google-generativeai==0.8.5
googleapis-common-protos==1.72.0
grpcio==1.76.0
grpcio-status==1.71.2
h11==0.16.0
httpcore==1.0.9
httplib2==0.31.0
httpx==0.28.1
idna==3.11
ipykernel==7.1.0
ipython==9.7.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
jupyter_client==8.6.3
jupyter_core==5.9.1
matplotlib-inline==0.2.1
nest-asyncio==1.6.0
packaging==25.0
parso==0.8.5
platformdirs==4.5.0
prompt_toolkit==3.0.52
proto-plus==1.26.1
protobuf==5.29.5
psutil==7.1.3
pure_eval==0.2.3
pyasn1==0.6.1
pyasn1_modules==0.4.2
pydantic==2.12.4
pydantic_core==2.41.5
Pygments==2.19.2
pyparsing==3.2.5
pyth

In [2]:
from google import genai
import os, getpass

In [3]:
# Obtener la API Key de entorno o pedirla al usuario de forma segura
API_KEY = os.getenv("GOOGLE_API_KEY") or getpass.getpass("Introduce tu API Key de Google Generative AI: ")

# Crear el cliente con la API Key (usando modo Vertex AI "express")
client = genai.Client(api_key=API_KEY, vertexai=True)


## 2. Generación simple variando `temperature` y `top_p`

Los modelos generativos permiten ajustar parámetros de muestreo para controlar la aleatoriedad y diversidad de las respuestas.  
Los dos parámetros más comunes son **temperature** (temperatura) y **top_p** (núcleo de probabilidad):

### 🔹 Temperature (`temperature`)

Controla la aletoriedad de elección de la siguiente palabra:

- Un valor bajo (por ejemplo, `0.2`) hace que el modelo sea más conservador y repetitivo.  
- Un valor alto (por ejemplo, `0.8`) lo hace más creativo o impredecible.  
- Una temperatura de `0` significa elegir siempre el token más probable (comportamiento casi determinista).

### 🔹 Top-p (`top_p`)

Define el porcentaje acumulado de probabilidad desde el cual el modelo elige las siguientes palabras.

- Por ejemplo, con `top_p = 0.5` el modelo solo considera las palabras cuya probabilidad acumulada suma el 50% y descarta el resto.  
- Un valor bajo limita la variedad (respuestas más seguras y predecibles).  
- Un valor cercano a `1` considera un rango más amplio de opciones, incrementando la diversidad.

---

Veamos un ejemplo sencillo variando estos parámetros.  
Usaremos el mismo prompt con distintas configuraciones de `temperature` y `top_p` para observar cómo cambia la respuesta.


In [4]:
prompt = "Escribe una breve nota sobre un gato que viaja al espacio."

# Respuesta con valores por defecto (temperature ~1.0, top_p ~0.95 por defecto)
resp_default = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
)

print("=== Respuesta (por defecto) ===")
print(resp_default.text)
print("\n" + "-"*50 + "\n")

# Respuesta con temperatura baja (más determinista) y top_p alto (considera más opciones pero con baja aleatoriedad)
resp_conservadora = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={"temperature": 0.2, "top_p": 1.0}
)

print("=== Respuesta (temperature=0.2) ===")
print(resp_conservadora.text)
print("\n" + "-"*50 + "\n")


# Respuesta con temperatura más alta (más creativa/aleatoria)
resp_creativa = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={"temperature": 0.8, "top_p": 1.0}
)

print("=== Respuesta (temperature=0.8) ===")
print(resp_creativa.text)
print("\n" + "-"*50 + "\n")




=== Respuesta (por defecto) ===
Aquí tienes una breve nota sobre un gato que viaja al espacio:

**Orion, El Astronauta Felino**

Orion no era un gato cualquiera. Era un atigrado pelirrojo, con ojos tan curiosos como las estrellas que a veces miraba desde la ventana. Un día, la curiosidad le ganó. Mientras los preparativos de una misión espacial estaban en pleno apogeo en el centro cercano, Orion encontró la oportunidad perfecta. Se coló, sigilosamente, en una de las naves justo antes del cierre de la escotilla final.

Los astronautas lo descubrieron flotando pacíficamente unas horas después del despegue, con una expresión de asombro en su pequeño rostro. Lejos de asustarse, Orion se adaptó con una facilidad sorprendente. Cazaba motas de polvo en gravedad cero, se acurrucaba en los regazos mientras los humanos trabajaban, y pasaba horas pegado a la escotilla, contemplando el azul mármol de la Tierra.

Se convirtió en la mascota no oficial de la misión, un faro de calidez y extraña norma

### Explicación

En el código anterior, enviamos el mismo texto de entrada al modelo **Gemini 2.5 Flash** con diferentes configuraciones.

* Con una **temperatura baja (`0.2`)**, la respuesta suele ser más predecible y similar en cada ejecución (poca aleatoriedad).
* Con una **temperatura alta (`0.8`)**, el modelo tiene más libertad creativa para variar la salida.

En este caso mantuvimos:

```
top_p = 1.0
```

Esto significa que **no recortamos el espacio de tokens por probabilidad acumulada**, para aislar únicamente el efecto de la temperatura.

Si redujéramos `top_p` a, por ejemplo, `0.5`, el modelo limitaría sus opciones solo a las palabras más probables hasta cubrir un 50% de probabilidad acumulada, obteniendo respuestas más conservadoras independientemente del valor de la temperatura.

---

### ¿Qué observarás al ejecutar el código?

Al ejecutar este código, podrás comparar las salidas generadas por el modelo:

* Con **temperatura baja**, el modelo podría responder con una nota muy sencilla y casi idéntica en cada ejecución.
* Con **temperatura alta**, podría introducir detalles imaginativos (por ejemplo, un *gato astronauta describiendo los colores de los planetas*).

Ajustando `temperature` y `top_p` puedes afinar el balance entre:

* ✅ **Coherencia**
* 🎨 **Creatividad**

según las necesidades de tu aplicación con modelos LLM.

---



## 3. Manejo de bloqueos por filtros de seguridad

Google Generative AI incorpora **filtros de seguridad** que pueden bloquear ciertas solicitudes o respuestas consideradas dañinas o inapropiadas.
Es importante manejar estos casos para evitar que nuestra aplicación falle inesperadamente y para respetar las políticas de uso.

---

### ¿Cuándo se bloquea una respuesta?

Una respuesta puede ser bloqueada si el contenido generado se considera:

* Ofensivo
* De incitación al odio
* De violencia extrema
* O de cualquier otra categoría insegura

Cuando esto ocurre:

* El modelo **no devuelve texto**,
* Sino que indica un motivo de finalización especial (`finishReason`).

Por ejemplo, si la respuesta fue filtrada por contenido inseguro, vendrá con:

```json
finishReason: "FINISH_REASON_SAFETY"
```

y sin contenido generado.

Como desarrolladores, debemos detectar este caso y actuar en consecuencia, por ejemplo:

* Mostrar un mensaje de advertencia al usuario.
* Evitar mostrar una respuesta vacía.
* Registrar el evento en logs para análisis futuros.

---

### Configuración de filtros con `safety_settings`

En el SDK, se pueden personalizar los filtros de seguridad a través de `safety_settings` dentro de la configuración del modelo.

En el siguiente ejemplo, forzaremos un bloqueo intencional para aprender a detectarlo:

* Pediremos al modelo que genere lenguaje hostil.
* Configuraremos el filtro para bloquear incluso acoso leve (`harassment`) con un umbral bajo.

Esto nos permitirá observar cómo responde el sistema cuando el filtro de seguridad se activa.


In [5]:
from google.genai import types

# Configurar filtro de seguridad: bloquear acoso de nivel bajo en adelante
safety_settings = [
    types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_LOW_AND_ABOVE"),
    types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_LOW_AND_ABOVE"),
    types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_LOW_AND_ABOVE"),
    types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_LOW_AND_ABOVE"),
]

# Solicitud deliberadamente ofensiva para disparar el filtro
respuesta = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Can you write a sarcastic rant about how the universe keeps hurting people for fun?",
    config=types.GenerateContentConfig(safety_settings=safety_settings)
)

# Verificar si la respuesta fue bloqueada
print(respuesta.candidates[0].finish_reason)
print("✔ Respuesta del modelo:", respuesta.text)


FinishReason.STOP
✔ Respuesta del modelo: Oh, *hello*, Universe! Don't mind me, just over here, trying to enjoy a *single* uninterrupted moment of peace before you decide it's time for your next knee-slapping comedy routine. Because apparently, that's what we are to you, isn't it? A colossal, multi-billion-year-old improv stage, and humanity is your endlessly flailing cast of clueless puppets.

Honestly, I'm starting to think you have a dartboard up there, adorned with pictures of perfectly content people, and your favorite cosmic pastime is just lobbing meteorites – or, you know, sudden financial crises, or inexplicably lost car keys, or that *one* tiny, impossible-to-reach itch – right at our heads. "Oh, look!" you probably cackle, spilling stardust all over your celestial couch, "This one was *just* about to achieve their dream! QUICK! Give them an acute allergy to oxygen!"

And the timing! Oh, your comedic timing is *impeccable*. Did I just finally get that promotion? Time for the 

In [6]:
respuesta.candidates[0]

Candidate(
  avg_logprobs=-1.8023059806326882,
  content=Content(
    parts=[
      Part(
        text="""Oh, *hello*, Universe! Don't mind me, just over here, trying to enjoy a *single* uninterrupted moment of peace before you decide it's time for your next knee-slapping comedy routine. Because apparently, that's what we are to you, isn't it? A colossal, multi-billion-year-old improv stage, and humanity is your endlessly flailing cast of clueless puppets.

Honestly, I'm starting to think you have a dartboard up there, adorned with pictures of perfectly content people, and your favorite cosmic pastime is just lobbing meteorites – or, you know, sudden financial crises, or inexplicably lost car keys, or that *one* tiny, impossible-to-reach itch – right at our heads. "Oh, look!" you probably cackle, spilling stardust all over your celestial couch, "This one was *just* about to achieve their dream! QUICK! Give them an acute allergy to oxygen!"

And the timing! Oh, your comedic timing is *i

### Explicación: bloqueo por filtros de seguridad

Solicitamos deliberadamente una salida inapropiada (por ejemplo, *"algo muy insultante"*) y establecemos en `safety_settings` que se bloquee cualquier contenido de acoso, incluso de baja intensidad.

El modelo intentará seguir la instrucción, pero el filtro interceptará la respuesta antes de retornarla. En consecuencia:

- `respuesta.text` estará vacío o será `null`
- `respuesta.finish_reason` indicará `"SAFETY"` para señalar que fue detenido por razones de seguridad

En el código, detectamos esta situación comprobando:

```python
if not respuesta.text or respuesta.finish_reason == "SAFETY":
    print("La respuesta fue bloqueada por filtros de seguridad")


## 4. Salida en formato JSON estructurado

A veces nos interesa que el modelo devuelva datos estructurados (por ejemplo, en formato **JSON**) en lugar de texto libre, para facilitar su procesamiento automático.

Los modelos **Gemini** pueden adaptar sus respuestas a un esquema JSON dado, lo que garantiza una salida con formato predecible y fácil de parsear.  
Esto es especialmente útil para tareas como:

- Extracción de información
- Clasificación estructurada
- Integración con otras herramientas

---

### Uso de JSON Schema con el SDK

El SDK de **Google GenAI** permite especificar un **JSON Schema** para la respuesta generada.

Podemos definir este esquema usando, por ejemplo, **Pydantic** en Python (una biblioteca para crear modelos de datos).

El flujo general es:

1. Definimos una clase en Pydantic con los campos que queremos.
2. Se la pasamos al modelo como requisito de formato.
3. El modelo genera un JSON siguiendo ese esquema.
4. Validamos el JSON de salida contra la clase para obtener un objeto Python tipado.

---

### Ejemplo de uso

Supongamos que queremos extraer información de una frase sobre una persona.

Definiremos un esquema con los siguientes campos:

- `nombre`
- `profesión`
- `edad`
- `país`

Luego pediremos al modelo que extraiga esos datos a partir de un texto dado.


In [7]:
from pydantic import BaseModel, Field

# Definir el esquema de datos usando Pydantic
class PersonaInfo(BaseModel):
    nombre: str = Field(description="Nombre de la persona")
    profesion: str = Field(description="Profesión u oficio")
    edad: int = Field(description="Edad en años")
    pais: str = Field(description="País de origen o residencia")

# Texto de ejemplo del cual extraer información
texto = "Carlos es un desarrollador de software de 29 años, originario de España."

# Pedir al modelo que extraiga la información en formato JSON según el esquema
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"Extrae los datos de la persona en formato JSON.\nTexto: \"{texto}\"",
    config={
        "response_mime_type": "application/json",
        "response_json_schema": PersonaInfo.model_json_schema()
    }
)

print("Salida JSON bruto:", response.text)
# Validar y convertir el JSON a objeto Pydantic
persona = PersonaInfo.model_validate_json(response.text)
print("Objeto PersonaInfo:", persona)
print("Nombre:", persona.nombre, "- Profesión:", persona.profesion)


Salida JSON bruto: {
  "nombre": "Carlos",
  "profesion": "desarrollador de software",
  "edad": 29,
  "pais": "España"
}
Objeto PersonaInfo: nombre='Carlos' profesion='desarrollador de software' edad=29 pais='España'
Nombre: Carlos - Profesión: desarrollador de software


### Explicación: generación y validación de salida JSON con Pydantic

Definimos la clase `PersonaInfo` que extiende `BaseModel` de **Pydantic**, con los campos que queremos extraer. Cada campo tiene un tipo (`str` o `int`) y una descripción opcional.

Le damos al modelo un texto de entrada sobre **"Carlos"** y le solicitamos: *"Extrae los datos de la persona en formato JSON"*. Incluimos el texto original para que el modelo lo analice.

En `config`, especificamos:

```json
"response_mime_type": "application/json"
```

y proporcionamos:

```
"response_json_schema": PersonaInfo.model_json_schema()
```

Este método genera un diccionario que describe el **JSON Schema** esperado a partir del modelo Pydantic.

El modelo devolverá un texto que es un **JSON válido** con los campos `nombre`, `profesion`, `edad` y `pais` que coincidan con la información del texto. Por ejemplo:

```json
{"nombre": "Carlos", "profesion": "desarrollador de software", "edad": 29, "pais": "España"}
```

Imprimimos la salida JSON cruda (`response.text`) para ver el string generado. Luego usamos:

```python
PersonaInfo.model_validate_json(response.text)
```

para parsearlo y validarlo contra nuestro modelo Pydantic.

Si algún dato no encaja con el esquema (por ejemplo, `edad` no es un número), esta validación lanzará un error. Idealmente, si el modelo siguió bien el formato, la validación será exitosa.

Finalmente, mostramos el objeto `persona` resultante y accedemos a sus campos usando atributos Python:

```python
persona.nombre
persona.profesion
persona.edad
persona.pais
```

---

Este enfoque garantiza que la respuesta del modelo sea **estructurada**. Nos ahorra tener que hacer parsing manual con expresiones regulares u otras heurísticas, y nos da confianza de que los campos esperados estarán presentes (o sabremos si falta alguno).

Para mejores resultados:

* Describe claramente cada campo en el schema (usando `description`).
* Si la tarea es compleja, detalla también el formato esperado en el prompt.

En aplicaciones más avanzadas, los **JSON Schema** pueden volverse más complejos (anidación, listas, tipos opcionales), pero los modelos Gemini suelen ser capaces de seguirlos siempre que la solicitud esté bien formulada.


## 5. Uso de sesiones de chat con persistencia de contexto

Además de generar texto a partir de un prompt aislado, el SDK soporta sesiones de chat que mantienen el contexto entre turnos, similar a conversar con ChatGPT u otros asistentes. Esto es útil para diálogos multi-turno donde el modelo debe recordar lo dicho anteriormente y responder acorde.

Con `genai.Client`, podemos crear una sesión de chat y luego enviar mensajes secuencialmente. El contexto (historial de mensajes) se conserva en el objeto de chat, así que el modelo recibe de forma implícita lo que se habló antes.

Veamos un ejemplo:


In [8]:
# Crear una nueva sesión de chat con un modelo de lenguaje
chat = client.chats.create(model="gemini-2.5-flash")

# Primer mensaje del usuario en la conversación
respuesta1 = chat.send_message("Hola, necesito que me recomiendes 3 ciudades de España para visitar.")
print("Asistente:", respuesta1.text)
print("\n" + "-"*50 + "\n")
print("\n" + "-"*50 + "\n")

# Segundo mensaje del usuario, haciendo referencia a la respuesta previa
respuesta2 = chat.send_message("Gracias. De esas ciudades, ¿cuál tiene la mayor población actualmente?")
print("Asistente:", respuesta2.text)
print("\n" + "-"*50 + "\n")
print("\n" + "-"*50 + "\n")


Asistente: ¡Claro que sí! España es un país con una riqueza cultural y paisajística increíble, y elegir solo tres ciudades es difícil, pero te daré tres opciones muy diferentes y con mucho que ofrecer para distintos gustos:

1.  **Madrid:**
    *   **Por qué visitarla:** Es la capital y el corazón cultural de España. Madrid es vibrante, cosmopolita y llena de energía. Aquí encontrarás museos de talla mundial (El Prado, Reina Sofía, Thyssen-Bornemisza), imponentes palacios (Palacio Real), hermosos parques (Parque del Retiro) y una vida nocturna inigualable. La gastronomía es excepcional, con infinidad de bares de tapas y restaurantes de todo tipo. Es una ciudad que nunca duerme y que ofrece una auténtica inmersión en la vida española.
    *   **Ideal para:** Amantes del arte, la historia, la buena comida, las compras y la vida urbana.

2.  **Sevilla:**
    *   **Por qué visitarla:** La capital de Andalucía es pura pasión y encanto. Sevilla te transportará con su impresionante arquitectu

### Explicación: sesiones de chat con contexto

Iniciamos una sesión con `client.chats.create(...)`. Luego usamos `chat.send_message(...)` para enviar mensajes de usuario a esa conversación.

En este ejemplo:

* El usuario saluda y pide una recomendación de **3 ciudades españolas para visitar**.
  El asistente (el modelo) podría responder algo como:
  *"Claro, algunas ciudades populares son Madrid, Barcelona y Sevilla, por sus atractivos turísticos."*
  (el contenido exacto lo determinará el modelo).

* En la segunda interacción, el usuario dice:
  *"Gracias. De esas ciudades, ¿cuál tiene la mayor población actualmente?"*

Nótese que el usuario no repite las ciudades por nombre, sino que se refiere a **"esas ciudades"**, asumiendo que el modelo recuerda las mencionadas anteriormente.

Como usamos el mismo objeto `chat`, el modelo tiene el contexto de la primera respuesta. Por lo tanto, puede inferir que se refiere a **Madrid, Barcelona y Sevilla** y contestar algo consistente, por ejemplo:

> *"Actualmente, Madrid es la ciudad con mayor población de las tres"*.

Imprimimos las respuestas del asistente después de cada turno.

---

En una sesión de chat, el SDK se encarga de enviar el **historial completo** en cada nueva pregunta, por lo que el modelo recuerda el contexto automáticamente.

Podemos seguir llamando `chat.send_message(...)` para continuar la conversación tantas veces como queramos. Esto facilita implementar asistentes conversacionales donde el modelo mantiene la memoria de lo conversado.

Ten en cuenta que existen límites de tokens (contexto máximo que el modelo puede recordar), pero modelos como **Gemini Pro** y **Gemini Flash** manejan contextos bastante amplios comparados con generaciones anteriores.


## 6. Chain-of-Thought prompting (respuesta directa vs paso a paso)

El término **Chain-of-Thought** (Cadena de Pensamiento) se refiere a una técnica de *prompting* donde animamos al modelo a **pensar paso a paso** antes de dar una respuesta final.

En lugar de responder directamente, el modelo expone su **razonamiento intermedio**, lo cual a menudo conduce a respuestas más precisas en problemas complejos.

Podemos lograr esto agregando indicaciones en el prompt del estilo:

- *"Pensemos paso a paso"*
- *"Analiza cuidadosamente antes de responder"*
- *"Muestra tu razonamiento antes de la respuesta final"*

Para ilustrar la diferencia, consideremos una pregunta tipo acertijo o de lógica.  
Haremos que el modelo responda:

1. De forma normal (respuesta directa).  
2. Con una indicación de **cadena de pensamiento** (*Chain-of-Thought*).


In [9]:
pregunta = "Si 5 máquinas hacen 5 artículos en 5 minutos, ¿cuánto tiempo tardarán 100 máquinas en hacer 100 artículos?"

# Respuesta directa (sin indicación de pensamiento paso a paso)
resp_directa = client.models.generate_content(
    model="gemini-2.0-flash-lite-001",
    contents=pregunta
)
print("Respuesta directa:", resp_directa.text)

# Respuesta con indicación de razonamiento paso a paso
prompt_cot = "Pensemos paso a paso:\n" + pregunta
resp_cadena = client.models.generate_content(
    model="gemini-2.0-flash-lite-001",
    contents=prompt_cot
)
print("Respuesta con chain-of-thought:", resp_cadena.text)


Respuesta directa: Si 5 máquinas hacen 5 artículos en 5 minutos, esto significa que cada máquina tarda 5 minutos en hacer un artículo. 

Si tenemos 100 máquinas, y cada máquina puede hacer un artículo en 5 minutos, entonces las 100 máquinas también harán 100 artículos en **5 minutos**.

Respuesta con chain-of-thought: Aquí está cómo resolver el problema paso a paso:

1. **Análisis inicial:** Si 5 máquinas hacen 5 artículos en 5 minutos, esto significa que cada máquina, en promedio, hace un artículo en 5 minutos.  (Una forma más fácil de pensar esto es que cada máquina necesita 5 minutos para hacer un artículo.)

2. **Escalado:** Si tienes 100 máquinas, y cada máquina puede hacer un artículo en 5 minutos, entonces las 100 máquinas pueden hacer 100 artículos en el mismo tiempo.

3. **Conclusión:**  Las 100 máquinas tardarán **5 minutos** en hacer 100 artículos.



### Explicación: Chain-of-Thought aplicado a un acertijo

La pregunta es un acertijo donde la respuesta correcta está implícita en la premisa ("El quinto hijo es Juan", ya que se dice *el padre de Juan*).

Sin contexto adicional, el modelo podría dar la respuesta directamente. Muchos LLMs entrenados reconocen este acertijo y responden **"Juan"** inmediatamente.

En la segunda versión, preparamos el prompt con *"Pensemos paso a paso:"*. Esto suele provocar que el modelo enumere sus deducciones antes de llegar a la conclusión. Por ejemplo, podría generar algo como:

> "Primero, el enunciado dice que el padre de Juan tiene 5 hijos. Se mencionan cuatro nombres: Ana, Bruno, Carlos y Daniel. Como son 5 en total y ya tenemos 4 nombres, el quinto hijo debe ser el propio Juan. Respuesta: Juan".

Aquí el modelo recorre explícitamente la lógica del problema antes de dar la solución.

---

Esta técnica de **Chain-of-Thought** es útil cuando queremos que el modelo muestre su proceso de razonamiento (por ejemplo, para entender por qué responde algo) o cuando trabajamos con problemas complejos de matemáticas o lógica.

Pedirle al modelo que *piense paso a paso* a menudo mejora la exactitud de la respuesta final, porque le da una estructura para organizar sus cálculos o argumentos en lugar de intentar dar una solución instantánea.

Ten en cuenta que la cadena de pensamiento normalmente se muestra en la respuesta del modelo. Si quieres que el modelo la use solo internamente pero no la muestre, necesitarías post-procesar la salida o usar técnicas más avanzadas (como funciones o herramientas, si están disponibles).

En este contexto educativo, resulta valioso comparar ambas modalidades:

* ✅ Respuesta directa
* 🧠 Respuesta con razonamiento paso a paso


# Ejercicio: Consulta con múltiples candidatos en Gemini API

Realiza una consulta a un modelo Gemini configurando el parámetro `candidate_count` para obtener varias respuestas alternativas a la vez (por ejemplo, 2 o 3).  
- Elige un prompt de tu interés.
- Muestra todas las respuestas generadas y comenta brevemente sus diferencias.


In [10]:
prompt = (
    "Eres un asistente creativo especializado en ideas de apps para startups. "
    "Responde en formato JSON con los campos 'nombre' y 'descripción', usando tono divertido y para público infantil. "
    "Dame tres ideas de nombres para una app de recetas."
)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.8,
        max_output_tokens=128,
        candidate_count=2,  
        top_p=0.9,
        top_k=40,
        response_mime_type='application/json'
    ),
)
# Para mostrar todos los candidatos:
for idx, candidate in enumerate(response.candidates, 1):
    print(f"\n--- Candidato {idx} ---\n{candidate.content.parts[0].text}")




--- Candidato 1 ---
[
  {
    "nombre": "¡Cocina Mágica de Mini Chefs!",
    "descripción": "¡Abracadabra, comida deliciosa! Aprende recetas súper fáciles y divertidas con ingredientes que parecen sacados de un cuento de hadas. ¡Cada plato es una aventura!"
  },
  {
    "nombre": "Los Pequeños Sabores Explosivos",
    "descripción": "¡Prepárate para una explosión de sabor! Descubre recetas secretas que harán bailar a tus papilas gustativas. ¡Cocinar nunca fue tan emocionante y lleno de

--- Candidato 2 ---
[
  {
    "nombre": "¡Ñam Ñam Recetas Mágicas!",
    "descripción": "¡A cocinar se ha dicho! Con esta app, aprenderás recetas superdivertidas con ingredientes secretos que harán ¡magia en tu pancita! ¡Prepárate para convertirte en un chef estrella!"
  },
  {
    "nombre": "Cocinitas Aventura",
    "descripción": "¡Únete a Cocinitas en una aventura culinaria! Descubre recetas secretas escondidas en el bosque de los vegetales y el volcán de chocolate


# Ejercicio: Chat y seguimiento de contexto con Gemini API

Vas a crear una simulación de chat entre un usuario y un modelo de Gemini, utilizando la interfaz de chat de la API.

## Instrucciones

1. Inicializa una sesión de chat con un modelo Gemini (elige un modelo que soporte conversación, como `gemini-2.0-flash` o `gemini-2.5-flash`).
2. Realiza **tres interacciones** consecutivas, donde cada mensaje del usuario depende del anterior (por ejemplo, pide primero información general, luego una aclaración o un ejemplo, y finalmente una petición concreta relacionada con los mensajes previos).
3. En la **tercera interacción**, plantea una pregunta que obligue al modelo a referirse explícitamente al contexto o detalles de la conversación anterior (por ejemplo, “¿Puedes resumir lo que hemos hablado hasta ahora?” o “Basándote en lo que me dijiste antes, ¿qué recomendarías?”).
4. Muestra el historial completo del chat y las respuestas del modelo.

## Reflexión

- ¿El modelo fue capaz de mantener el contexto y dar respuestas coherentes?
- ¿Observas alguna limitación o pérdida de información entre turnos?
- ¿Qué técnicas o configuraciones podrían mejorar la memoria conversacional del modelo?


In [11]:
chat = client.chats.create(model='gemini-2.0-flash')

In [12]:
response = chat.send_message(
    message="Eres un experto chef que responde solo con recetas rápidas y en tono divertido.Quiero una receta rápida con arroz.",
    config=types.GenerateContentConfig(
        temperature=0.6,
        max_output_tokens=150),
)
print("Asistente:", response.text)

Asistente: ¡Órale, mi estrellita Michelin en pañales! ¿Antojo de arroz rápido y sabrosón? ¡Aquí te va un "Arroz a la Diabla Express" que te hará sudar hasta las ideas!

**Ingredientes:**

*   1 taza de arroz blanco precocido (¡si no, te doy con la cuchara!)
*   1 cucharada de aceite (¡del que no te haga toser!)
*   1/2 cebolla picada (¡llora si quieres, pero pícala bien!)
*   1 diente de ajo picado (¡que no te dé miedo el aliento!)
*   1 lata de tomate frito (¡del que te guste


In [13]:
response = chat.send_message(
    message="¿Y ahora una receta con pasta?",
    config=types.GenerateContentConfig(
        temperature=0.6,
        max_output_tokens=150),
)
print("Asistente:", response.text)

Asistente: ¡Pasta, dices! ¡Agárrate que vamos a cocinar con turbo! Aquí te va una "Pasta Aglio e Olio Picante Relámpago" que te dejará pidiendo más en menos de lo que canta un gallo italiano:

**Ingredientes:**

*   200g de espagueti (¡o la pasta que te baile en el plato!)
*   4 cucharadas de aceite de oliva (¡del bueno, que no somos tacaños!)
*   4 dientes de ajo laminados (¡como si fueran monedas de oro!)
*   1 guindilla (¡o chile seco, si te va la marcha!)
*   Perejil fresco picado (


In [14]:
response = chat.send_message(
    message="¿Te acuerdas de que eran las dos recetas que te he pedido?",
    config=types.GenerateContentConfig(
        temperature=0.6,
        max_output_tokens=150),
)
print("Asistente:", response.text)

Asistente: ¡Claro que sí, mi memoria es mejor que la de un robot de cocina! 😉

1.  **Arroz a la Diabla Express:** La receta de arroz picante y rapidita que te haría sudar hasta las ideas.
2.  **Pasta Aglio e Olio Picante Relámpago:** La receta de pasta con ajo, aceite y guindilla que te dejaba pidiendo más en menos de lo que canta un gallo italiano.

¿Necesitas que te recuerde algún ingrediente o paso en particular? ¡Dispara, que estoy aquí para eso!



In [15]:
for idx, turn in enumerate(chat.get_history(), 1):
    rol = turn.role
    # Accede al texto de la respuesta; a veces turn.parts es una lista con .text
    texto = turn.parts[0].text if hasattr(turn.parts[0], 'text') else str(turn.parts[0])
    print(f"{rol}: {texto}")

user: Eres un experto chef que responde solo con recetas rápidas y en tono divertido.Quiero una receta rápida con arroz.
model: ¡Órale, mi estrellita Michelin en pañales! ¿Antojo de arroz rápido y sabrosón? ¡Aquí te va un "Arroz a la Diabla Express" que te hará sudar hasta las ideas!

**Ingredientes:**

*   1 taza de arroz blanco precocido (¡si no, te doy con la cuchara!)
*   1 cucharada de aceite (¡del que no te haga toser!)
*   1/2 cebolla picada (¡llora si quieres, pero pícala bien!)
*   1 diente de ajo picado (¡que no te dé miedo el aliento!)
*   1 lata de tomate frito (¡del que te guste
user: ¿Y ahora una receta con pasta?
model: ¡Pasta, dices! ¡Agárrate que vamos a cocinar con turbo! Aquí te va una "Pasta Aglio e Olio Picante Relámpago" que te dejará pidiendo más en menos de lo que canta un gallo italiano:

**Ingredientes:**

*   200g de espagueti (¡o la pasta que te baile en el plato!)
*   4 cucharadas de aceite de oliva (¡del bueno, que no somos tacaños!)
*   4 dientes de ajo l